<a href="https://colab.research.google.com/github/run-llama/llama_index/blob/main/docs/examples/low_level/oss_ingestion_retrieval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Building RAG from Scratch (Open-source only!)

In this tutorial, we show you how to build a data ingestion pipeline into a vector database, and then build a retrieval pipeline from that vector database, from scratch.

Notably, we use a fully open-source stack:

- Sentence Transformers as the embedding model
- Postgres as the vector store (we support many other [vector stores](https://gpt-index.readthedocs.io/en/stable/core_modules/data_modules/storage/vector_stores.html) too!)
- Llama 2 as the LLM (through [llama.cpp](https://github.com/ggerganov/llama.cpp))

## Setup

We setup our open-source components.
1. Sentence Transformers
2. Llama 2
3. We initialize postgres and wrap it with our wrappers/abstractions.

#### Sentence Transformers

In [1]:
# sentence transformers
from llama_index.embeddings import HuggingFaceEmbedding

embed_model = HuggingFaceEmbedding(model_name="BAAI/bge-small-en")

#### Llama CPP

In this notebook, we use the [`llama-2-chat-13b-ggml`](https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML) model, along with the proper prompt formatting.

Check out our [Llama CPP guide](https://gpt-index.readthedocs.io/en/stable/examples/llm/llama_2_llama_cpp.html) for full setup instructions/details.

In [2]:
from llama_index.llms import LlamaCPP

# model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGML/resolve/main/llama-2-13b-chat.ggmlv3.q4_0.bin"
model_url = "https://huggingface.co/TheBloke/Llama-2-13B-chat-GGUF/resolve/main/llama-2-13b-chat.Q4_0.gguf"

llm = LlamaCPP(
    # You can pass in the URL to a GGML model to download it automatically
    model_url=model_url,
    # optionally, you can set the path to a pre-downloaded model instead of model_url
    model_path=None,
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    verbose=True,
)

llama_model_loader: loaded meta data with 19 key-value pairs and 363 tensors from /Users/richy/Library/Caches/llama_index/models/llama-2-13b-chat.Q4_0.gguf (version GGUF V2)
llama_model_loader: - tensor    0:                token_embd.weight q4_0     [  5120, 32000,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q4_0     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q4_0     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q4_0     [  5120,  5120,     1,     1 ]
llama_model_loader: - tensor    7:         blk.0.attn_output.weight q4

#### Define Service Context

In [3]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

#### Initialize Postgres

Using an existing postgres running at localhost, create the database we'll be using.

**NOTE**: Of course there are plenty of other open-source/self-hosted databases you can use! e.g. Chroma, Qdrant, Weaviate, and many more. Take a look at our [vector store guide](https://gpt-index.readthedocs.io/en/stable/core_modules/data_modules/storage/vector_stores.html).

**NOTE**: You will need to setup postgres on your local system. Here's an example of how to set it up on OSX: https://www.sqlshack.com/setting-up-a-postgresql-database-on-mac/.

**NOTE**: You will also need to install pgvector (https://github.com/pgvector/pgvector).

You can add a role like the following:
```
CREATE ROLE <user> WITH LOGIN PASSWORD '<password>';
ALTER ROLE <user> SUPERUSER;
```

In [77]:
import psycopg2

db_name = "vector_db"
host = "localhost"
password = "123"
port = "5432"
user = "richyc"
# conn = psycopg2.connect(connection_string)
conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")

In [88]:
from sqlalchemy import make_url
from llama_index.vector_stores import PGVectorStore

vector_store = PGVectorStore.from_params(
    database=db_name,
    host=host,
    password=password,
    port=port,
    user=user,
    table_name="llama2_paper",
    embed_dim=384,  # openai embedding dimension
)

## Build an Ingestion Pipeline from Scratch

We show how to build an ingestion pipeline as mentioned in the introduction.

We fast-track the steps here (can skip metadata extraction). More details can be found [in our dedicated ingestion guide](https://gpt-index.readthedocs.io/en/latest/examples/low_level/ingestion.html).

### 1. Load Data

In [79]:
!pip install nest_asyncio

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [80]:
from llama_index import download_loader
import nest_asyncio
nest_asyncio.apply()

AsyncWebPageReader = download_loader("AsyncWebPageReader")

In [81]:
loader = AsyncWebPageReader()
documents = loader.load_data(urls=['https://insurify.com/car-insurance'])

### 2. Use a Text Splitter to Split Documents

In [82]:
from llama_index.text_splitter import SentenceSplitter

In [83]:
text_splitter = SentenceSplitter(
    chunk_size=1024,
    # separator=" ",
)

In [84]:
text_chunks = []
# maintain relationship with source doc index, to help inject doc metadata in (3)
doc_idxs = []
for doc_idx, doc in enumerate(documents):
    cur_text_chunks = text_splitter.split_text(doc.text)
    text_chunks.extend(cur_text_chunks)
    doc_idxs.extend([doc_idx] * len(cur_text_chunks))

### 3. Manually Construct Nodes from Text Chunks

In [85]:
from llama_index.schema import TextNode

nodes = []
for idx, text_chunk in enumerate(text_chunks):
    node = TextNode(
        text=text_chunk,
    )
    src_doc = documents[doc_idxs[idx]]
    node.metadata = src_doc.metadata
    nodes.append(node)

### 4. Generate Embeddings for each Node

Here we generate embeddings for each Node using a sentence_transformers model.

In [86]:
for node in nodes:
    node_embedding = embed_model.get_text_embedding(
        node.get_content(metadata_mode="all")
    )
    node.embedding = node_embedding

### 5. Load Nodes into a Vector Store

We now insert these nodes into our `PostgresVectorStore`.

Note: if vector extension does not exist, run `brew install pgvector` and then `CREATE EXTENSION IF NOT EXISTS vector` in postgres.

In [87]:
vector_store.add(nodes)

['39fb6374-1ec6-4f9e-9dc4-01ee22b752e9',
 '570d9e42-66af-45df-95ba-b6bb23efce16',
 '994c63fd-451d-4995-9596-a46a0a5047a7',
 '5a135de6-f92a-462c-9390-9854149fb952',
 '45c7fe1c-a390-4c49-98c6-218c60e38145',
 '4485e964-f7a0-45da-832b-0316774bf1db',
 'e0ec5d5a-051d-4bb3-bb03-ea5258a848fd',
 'adb3ea1f-9185-4a47-b278-131e3aa3bab2',
 'e35432ef-3fc2-456f-a6ce-5132a2c5f94c',
 '64606823-211b-4791-9322-a7d9caa6ae1b',
 'bea3f04c-613c-49f8-aa01-1613c7aaa0d5',
 '05b7fc00-e2e4-4b33-be5e-64d876d0faab',
 '57599ee4-1c2c-4ca5-9de1-75229a27b5ae',
 '02b88156-1358-499d-b62b-d8f78d3f7478',
 'b03dc1b4-935e-41a6-8edd-513d975cdf31',
 '09c6c608-76d2-4696-ab9f-8766f8ac6462',
 'fd005988-2114-4123-a47e-ccce11e77632',
 '10ed4b92-6741-4920-8023-e22353400bb3',
 '191a756d-48a9-443a-9ddf-da22335b9385',
 '80ba508c-3fb8-471f-bff4-816bf0edd48f',
 'd6a712b4-85d5-403b-801e-0e1aa48850a0',
 'a56d5dc0-db15-4501-8ca2-42af268ee42a',
 '56721c90-f793-4537-976b-1e2df6456953',
 'a47bf8e2-f25f-4b6b-a9ec-1c78f52533cb',
 '56ba97a4-b086-

## Build Retrieval Pipeline from Scratch

We show how to build a retrieval pipeline. Similar to ingestion, we fast-track the steps. Take a look at our [retrieval guide](https://gpt-index.readthedocs.io/en/latest/examples/low_level/retrieval.html) for more details!

In [59]:
query_str = "Can you tell me about the types of car insurance coverage?"

### 1. Generate a Query Embedding

In [60]:
query_embedding = embed_model.get_query_embedding(query_str)

### 2. Query the Vector Database

In [89]:
# construct vector store query
from llama_index.vector_stores import VectorStoreQuery

query_mode = "default"
# query_mode = "sparse"
# query_mode = "hybrid"

vector_store_query = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=2, mode=query_mode
)

In [90]:
# returns a VectorStoreQueryResult
query_result = vector_store.query(vector_store_query)
print(query_result.nodes[0].get_content())

It’s typically just enough coverage to meet your state’s minimum requirements, and nothing more. It provides essential financial protection for bodily injury and property damage liability if you’re in an accident. When you choose this minimum coverage, you don’t get protection for your own vehicle.</p><p>A liability-only policy may make sense if the cost to repair or replace your vehicle doesn’t justify the additional expense of full coverage, or if you’re on a tight budget and prioritize lower premiums over comprehensive protection. It may also be suitable if you frequently use public transportation and don’t spend much time behind the wheel.</p></div></div></li><li><div class="dC_yF"><img alt="illustration card https://a.storyblok.com/f/162273/x/38dc81ba93/drive-1.svg" loading="lazy" width="48" height="48" src="https://a.storyblok.com/f/162273/x/38dc81ba93/drive-1.svg/m/48x48/smart/"/><div><h4>Full coverage: Good for drivers who want more coverage for their own vehicle</h4><p><a href

### 3. Parse Result into a Set of Nodes

In [63]:
from llama_index.schema import NodeWithScore
from typing import Optional

nodes_with_scores = []
for index, node in enumerate(query_result.nodes):
    score: Optional[float] = None
    if query_result.similarities is not None:
        score = query_result.similarities[index]
    nodes_with_scores.append(NodeWithScore(node=node, score=score))

### 4. Put into a Retriever

In [64]:
from llama_index import QueryBundle
from llama_index.retrievers import BaseRetriever
from typing import Any, List


class VectorDBRetriever(BaseRetriever):
    """Retriever over a postgres vector store."""

    def __init__(
        self,
        vector_store: PGVectorStore,
        embed_model: Any,
        query_mode: str = "default",
        similarity_top_k: int = 2,
    ) -> None:
        """Init params."""
        self._vector_store = vector_store
        self._embed_model = embed_model
        self._query_mode = query_mode
        self._similarity_top_k = similarity_top_k

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        """Retrieve."""
        query_embedding = embed_model.get_query_embedding(query_str)
        vector_store_query = VectorStoreQuery(
            query_embedding=query_embedding,
            similarity_top_k=self._similarity_top_k,
            mode=self._query_mode,
        )
        query_result = vector_store.query(vector_store_query)

        nodes_with_scores = []
        for index, node in enumerate(query_result.nodes):
            score: Optional[float] = None
            if query_result.similarities is not None:
                score = query_result.similarities[index]
            nodes_with_scores.append(NodeWithScore(node=node, score=score))

        return nodes_with_scores

In [65]:
retriever = VectorDBRetriever(
    vector_store, embed_model, query_mode="default", similarity_top_k=2
)

## Plug this into our RetrieverQueryEngine to synthesize a response

In [66]:
from llama_index.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(
    retriever, service_context=service_context
)

In [67]:
query_str = "Can you tell me about the types of car insurance coverage?"

response = query_engine.query(query_str)

Llama.generate: prefix-match hit

llama_print_timings:        load time =   18266.67 ms
llama_print_timings:      sample time =      13.55 ms /   103 runs   (    0.13 ms per token,  7600.92 tokens per second)
llama_print_timings: prompt eval time =   37336.94 ms /  1991 tokens (   18.75 ms per token,    53.33 tokens per second)
llama_print_timings:        eval time =    6782.37 ms /   102 runs   (   66.49 ms per token,    15.04 tokens per second)
llama_print_timings:       total time =   44385.95 ms


In [68]:
print(str(response))

 Sure! There are several types of car insurance coverage available, including liability-only, full coverage, uninsured or underinsured motorist coverage, medical payments coverage, and comprehensive and collision coverage. Each type of coverage provides different levels of protection for your vehicle and your financial assets in the event of an accident or other loss. It's important to carefully consider your needs and budget when selecting car insurance coverage to ensure you have the right level of protection for your specific situation.


In [70]:
print(response.source_nodes[0].metadata)

{'Source': 'https://insurify.com/car-insurance/'}


In [71]:
len(response.source_nodes)

2